In [3]:
#!/usr/bin/python
#-*- coding: utf-8 -*-

import pymongo as pym
import re

candidates = ['fillon', 'macron', 'le pen', 'hamon', 'melenchon']

# le (?i) rend la regex insensible a la casse. Il ne faut pas laisser de liste vide
# sinon l'algo va tirer n'importe quel tweet dans la base sans filtre particulier
negatives = {'fillon': '(?i)#penelopegate|#fillongate|#penelope|#fillondemission',
             'macron': '(?i)#stopmacron|#levraimacron|#lepionmacron|#macrongate|#hollandebis|#macrongirouette',
             'le pen': '(?i)#lepengate|#fngate',
             'hamon': '(?i)#bilalhamon|#plusjamaisps|#hamonpiègeacons',
             'melenchon': '(?i)'}

neutrals = {'fillon': '(?i)#confpressfillon|#conffillon',
            'macron': '(?i)#macronlyon',
            'le pen': '(?i)',
            'hamon': '(?i)',
            'melenchon': '(?i)'}

positives = {'fillon': '(?i)#stopchassealhomme|#fillonpresident|#projetfillon|#soutienfillon',
             'macron': '(?i)#teammacron|#lafranceenmarche',
             'hamon': '(?i)#hamonpresident|#avechamon',
             'le pen' : '(?i)#aunomdupeuple|#marine2017|#mlp2017',
             'melenchon': '(?i)#6erépublique|#6erepublique|#franceinsoumise|#jlm2017|#placeaupeuple|#CantStenchonTheMelenchon|#FI|#18mars2017'}

In [20]:
def getTweets(candidates, sentimentList, sentiment, limit=1000):
    '''Echelle de classification -1 Négatif 0 Neutre 1 Positif
    Critère de sélection d'un tweet : ne doit pas être un retweet, ne doit contenir le nom 
    que d'un seul candidat'''
    
    client = pym.MongoClient()
    client = pym.MongoClient('localhost', 27017)
    collection = client.tweet.tweet
    labelisedCollection = client.tweet.labelised
    
    count = 0
    
    for candidate in candidates:
        a_inserer = []
        sentiment_regex = re.compile((sentimentList[candidate]))
        print('sentiment {} :'.format(sentiment), sentimentList[candidate])
        
        # filtrage des retweets en amont
        corpus = collection.find(
            filter={'$and': [{'t_text': {'$not': re.compile("^rt @")}},
                             {'t_text': sentiment_regex}]},
            projection={'_id':False, 't_id':1, 't_text':1}, limit=limit)

        for t in corpus: 
            a_inserer.append({'text': t['t_text'], 'sentiment': sentiment, 'candidat': candidate,
                             't_id': t['t_id']})
        
        labelisedCollection.insert_many(a_inserer)
        print(len(a_inserer), 'insertions effectuees.')
        count += len(a_inserer)
        
    print(45 * '-')
    print(count, 'insertions au total.')

    # retrait des doublons
    print('Retrait des doublons...')
    duplicates = []
    removepipe = [{"$group":{"_id":"$t_id", "dups":{"$push":"$_id"},
                             "count":{"$sum":1}}},{"$match":{"count":{"$gt":1}}}]
    count = 0
    try :
        for doc in labelisedCollection.aggregate(removepipe):
            it = iter(doc['dups'])
            next(it)
            for id in it :
                count += 1
                duplicates.append(pym.DeleteOne({'_id':id}))
            if duplicates:    
                labelisedCollection.bulk_write(duplicates)    
    except:
        pass
    
    print(count, 'doublons retirés.')
    client.close()

In [24]:
getTweets(candidates, positives, 1, limit=300)

sentiment 1 : (?i)#fillon2017|#stopchassealhomme|#fillonpresident|#projetfillon|#soutienfillon
300 insertions effectuees.
sentiment 1 : (?i)#enmarche|#teammacron|#lafranceenmarche|#macron2017
300 insertions effectuees.
sentiment 1 : (?i)#aunomdupeuple|#marine2017|#mlp2017
300 insertions effectuees.
sentiment 1 : (?i)#hamon2017|#benoithamon|#hamonpresident|#avechamon
300 insertions effectuees.
sentiment 1 : (?i)#6erépublique|#franceinsoumise|#jlm2017|#placeaupeuple|#CantStenchonTheMelenchon|#FI|#18mars2017
300 insertions effectuees.
---------------------------------------------
1500 insertions au total.
Retrait des doublons...
1500 doublons retirés.


In [25]:
client = pym.MongoClient('localhost', 27017)
col = client.tweet.labelised

In [26]:
# nombre de lignes dans la base
print(col.count())

4333
